In [21]:
import pandas as pd

df = pd.read_excel("processed/msamogh-turn-durations.xlsx")
df["turn_durations"] = df["turn_durations"].apply(eval)
df.head()

,trajectory,prompt,turn_durations
0,"GazePath(submit_time=1648242251845, dst={'arri...",semi,"{'1': 32.579999923706055, '0': 109.39000129699..."
1,"GazePath(submit_time=1648242555322, dst={'area...",semi,"{'0': 329.4700012207031, '1': 229.160003662109..."
2,"GazePath(submit_time=1648242753253, dst={'area...",semi,"{'0': 234.7700023651123, '2': 454.209993362426..."
3,"GazePath(submit_time=1648242886080, dst={'area...",semi,"{'4': 146.0299949645996, '0': 134.259994506835..."
4,"GazePath(submit_time=1648243035969, dst={'area...",semi,"{'4': 331.1599998474121, '0': 346.060003280639..."


In [22]:
df["even_turn_duration"] = df["turn_durations"].apply(lambda x: sum([v for k, v in x.items() if int(k) % 2 == 0]))
df["odd_turn_duration"] = df["turn_durations"].apply(lambda x: sum([v for k, v in x.items() if int(k) % 2 != 0]))

In [23]:
df.head()

,trajectory,prompt,turn_durations,even_turn_duration,odd_turn_duration
0,"GazePath(submit_time=1648242251845, dst={'arri...",semi,"{'1': 32.579999923706055, '0': 109.39000129699...",158.440002,50.869999
1,"GazePath(submit_time=1648242555322, dst={'area...",semi,"{'0': 329.4700012207031, '1': 229.160003662109...",1701.849987,952.919998
2,"GazePath(submit_time=1648242753253, dst={'area...",semi,"{'0': 234.7700023651123, '2': 454.209993362426...",1213.449999,479.129992
3,"GazePath(submit_time=1648242886080, dst={'area...",semi,"{'4': 146.0299949645996, '0': 134.259994506835...",828.039989,298.469997
4,"GazePath(submit_time=1648243035969, dst={'area...",semi,"{'4': 331.1599998474121, '0': 346.060003280639...",1070.360001,213.900005


In [24]:
df[df["prompt"] == "semi"].describe()

,even_turn_duration,odd_turn_duration
count,152.000000,152.000000
mean,516.834869,266.572763
std,271.888450,223.079612
min,4.990000,15.949997
25%,348.297502,114.607499
50%,519.474998,217.174998
75%,658.205001,320.092499
max,1701.849987,1285.309999


In [25]:
df[df["prompt"] == "goal"].describe()

,even_turn_duration,odd_turn_duration
count,149.000000,149.000000
mean,583.403893,187.519127
std,299.958091,154.359030
min,24.099998,5.020000
25%,391.439997,79.700003
50%,576.300003,146.810001
75%,733.399994,241.650000
max,1750.649998,972.100002


In [27]:
df.groupby("prompt").agg({"even_turn_duration": "mean", "odd_turn_duration": "mean"})

,even_turn_duration,odd_turn_duration
prompt,,
goal,583.403893,187.519127
semi,516.834869,266.572763


In [28]:
df.groupby("prompt").agg({"even_turn_duration": "median", "odd_turn_duration": "median"})

,even_turn_duration,odd_turn_duration
prompt,,
goal,576.300003,146.810001
semi,519.474998,217.174998
